In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [3]:
def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)
    return images, labels

In [4]:
x_train, y_train = load_mnist('',kind='train')
x_test, y_test = load_mnist('',kind='t10k')

In [5]:
x_train = x_train / 255.0
x_test = x_test / 255.0

In [73]:
model = Sequential([
    tf.keras.Input(shape=(784,1)),
    tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
])
model.add(layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [74]:
model.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_2 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_15 (Flatten)         (None, 576)             

In [75]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [76]:
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.4963 - accuracy: 0.8190
Epoch 2/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.3207 - accuracy: 0.88390s - loss: 0
Epoch 3/10
1875/1875 [==============================] - 22s 12ms/step - loss: 0.2738 - accuracy: 0.8989
Epoch 4/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.2444 - accuracy: 0.9093
Epoch 5/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.2217 - accuracy: 0.9178
Epoch 6/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.2012 - accuracy: 0.9250
Epoch 7/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.1828 - accuracy: 0.9314
Epoch 8/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.1676 - accuracy: 0.9375
Epoch 9/10
1875/1875 [==============================] - 24s 13ms/step - loss: 0.1542 - accuracy: 0.9423
Epoch 10/10
1875/1875 [=============================

In [77]:
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=1,batch_size = 1)
print('\nTest accuracy:', test_acc)

10000/10000 [==============================] - 7s 662us/step - loss: 0.2768 - accuracy: 0.9125

Test accuracy: 0.9125000238418579
